<h1>Income Inequality prediction</h1>

## Index
1. [🧾Description](#Description)

## Description
The Problem of income being distributed in an **uneven** manner among a population is a growing problem in developing nations across the world.

With the rapid rise of **AI** and **worker automation**, this problem could continue to grow if steps are not taken to address the issue.

This project is an effort to create a **machine learning** model that predict whether an individual earns above or below a certain amount based on  **key population indicators** that effect the income level, which can help policymakers to better manage and avoid income inequality globally.

The target feature is `income_above_limit` which is a **Binary-class** variable.

The metric used for evaluation is `f1-score`.

---